# Imports

data sources: https://github.com/CSSEGISandData/COVID-19_Unified-Dataset documentation: https://github.com/OxCGRT/covid-policy-tracker/blob/master/documentation/codebook.md

In [1]:
import datetime
import pandas as pd
import numpy as np
import pyreadr
from sklearn.preprocessing import OneHotEncoder

In [20]:
data_lma=pd.read_csv('https://raw.githubusercontent.com/TristanBOOK/coronanet/master/coronanet/data/LMA_Data_10_semaines_and_Pol_fam.csv')

In [21]:
data_lma.head()

,ID,Date,PolicyFamily,PolicyType,value,Cases,Cases_New,Type,Source,J-28,J-21,J-14,J-7,J0,J7,J14,J21,J28,J35,J42
0,AD,2020-02-27,E,E3_diff,141399008,0.0,0.0,Deaths,JHU,0,0,0,0,0,0,0,0,3,15,25
1,AD,2020-02-28,E,E3_diff,-141399008,0.0,0.0,Deaths,JHU,0,0,0,0,0,0,0,0,3,16,26
2,AD,2020-03-02,H,H2_diff,1,0.0,0.0,Deaths,JHU,0,0,0,0,0,0,0,1,8,21,29
3,AD,2020-03-11,H,H6_diff,2,0.0,0.0,Deaths,JHU,0,0,0,0,0,0,1,14,23,33,37
4,AD,2020-03-13,C,C3_diff,1,0.0,0.0,Deaths,JHU,0,0,0,0,0,0,3,16,26,35,40


# Policies DF

In [2]:
policies= pyreadr.read_r('/Users/paulcharpentier/Desktop/Coronanet_files/Policy.RData')
policies_df = pd.DataFrame(policies['Policy'])
policies_df = policies_df.rename(columns={"ID": "Country"})

In [3]:
policies_df.head()

,Country,Date,PolicyType,PolicyValue,PolicyFlag,PolicyNotes,PolicySource
0,AD,2020-01-01,C1,0.0,NaN,NaN,OxCGRT
1,AD,2020-01-01,C2,0.0,NaN,NaN,OxCGRT
2,AD,2020-01-01,C3,0.0,NaN,NaN,OxCGRT
3,AD,2020-01-01,C4,0.0,NaN,NaN,OxCGRT
4,AD,2020-01-01,C5,0.0,NaN,NaN,OxCGRT


In [4]:
policies_df.set_index('PolicyType',inplace=True)
policies_df=policies_df.drop("I1",axis=0)
policies_df=policies_df.drop("I2",axis=0)
policies_df=policies_df.drop("I3",axis=0)
policies_df=policies_df.drop("I4",axis=0)
policies_df=policies_df.drop("IC",axis=0)
policies_df=policies_df.drop("ID",axis=0)

In [5]:
policies_df.head()

,Country,Date,PolicyValue,PolicyFlag,PolicyNotes,PolicySource
PolicyType,,,,,,
C1,AD,2020-01-01,0.0,NaN,NaN,OxCGRT
C2,AD,2020-01-01,0.0,NaN,NaN,OxCGRT
C3,AD,2020-01-01,0.0,NaN,NaN,OxCGRT
C4,AD,2020-01-01,0.0,NaN,NaN,OxCGRT
C5,AD,2020-01-01,0.0,NaN,NaN,OxCGRT


In [6]:
policies_df['Policy']=policies_df.index
policies_df.set_index('Date', inplace=True)

In [7]:
policies_df.head()

,Country,PolicyValue,PolicyFlag,PolicyNotes,PolicySource,Policy
Date,,,,,,
2020-01-01,AD,0.0,NaN,NaN,OxCGRT,C1
2020-01-01,AD,0.0,NaN,NaN,OxCGRT,C2
2020-01-01,AD,0.0,NaN,NaN,OxCGRT,C3
2020-01-01,AD,0.0,NaN,NaN,OxCGRT,C4
2020-01-01,AD,0.0,NaN,NaN,OxCGRT,C5


# Policies feature engineering

Check

In [8]:
policies_df[policies_df.Policy=='ID'].PolicyValue.unique()

array([], dtype=float64)

The following policy values are continuous and cannot be categorised (they represent amount of money), we transform them into a power of ten and only keep the exponent to give an idea of the amount at stake.
E3, E4, H4, H5 
the 'I' values are indices, and are dropped from the table

In [9]:
def logify (x):
    if x > 6:
        try:
            if x>0:
                y= int(np.log10(x))
            else:
                y=int(0)
        except:
            y=np.nan
    else:
        try:
            y = int(x)
        except:
            y=np.nan
    return y

In [10]:
pol_df=policies_df

In [11]:
pol_df['PolicyValue']=pol_df['PolicyValue'].apply(logify)
#pol_df.PolicyValue=pol_df.PolicyValue.to_string()

In [12]:
pol_df.head()

,Country,PolicyValue,PolicyFlag,PolicyNotes,PolicySource,Policy
Date,,,,,,
2020-01-01,AD,0.0,NaN,NaN,OxCGRT,C1
2020-01-01,AD,0.0,NaN,NaN,OxCGRT,C2
2020-01-01,AD,0.0,NaN,NaN,OxCGRT,C3
2020-01-01,AD,0.0,NaN,NaN,OxCGRT,C4
2020-01-01,AD,0.0,NaN,NaN,OxCGRT,C5


In [13]:
type(pol_df['PolicyValue'].unique()[3])

numpy.float64

In [14]:
def stringify (x):
    try:
        if x == np.nan:
            y = int(0)
        else:
            y = int(x)
    except:
        y = int(0)
    return str(y)

In [15]:
pol=pol_df
pol.PolicyValue=pol.PolicyValue.apply(stringify)

In [16]:
pol.head()

,Country,PolicyValue,PolicyFlag,PolicyNotes,PolicySource,Policy
Date,,,,,,
2020-01-01,AD,0,NaN,NaN,OxCGRT,C1
2020-01-01,AD,0,NaN,NaN,OxCGRT,C2
2020-01-01,AD,0,NaN,NaN,OxCGRT,C3
2020-01-01,AD,0,NaN,NaN,OxCGRT,C4
2020-01-01,AD,0,NaN,NaN,OxCGRT,C5


Check

In [17]:
pol.Policy.unique()

array(['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'E1', 'E2', 'E3',
       'E4', 'H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'I1D', 'I2D', 'I3D',
       'I4D', 'I4L', 'I4LD', 'M1'], dtype=object)

In [18]:
a=pol['Policy'].iloc[1000]+'_'+pol['PolicyValue'].iloc[10000]
a

'C1_2'

In [19]:
p=pol
p['TypeVal']=p['Policy']+'_'+p['PolicyValue']

In [20]:
p.TypeVal.values

array(['C1_0', 'C2_0', 'C3_0', ..., 'I4L_0', 'I4LD_1', 'M1_0'],
      dtype=object)

In [21]:
# Number of features
len(p.TypeVal.unique())

125

In [22]:
q=pd.get_dummies(p.TypeVal)

In [23]:
q['Country']=p['Country']

In [24]:
final_df=pd.DataFrame()
for c in q.Country.unique():
    r=q[q['Country']==c].groupby('Date').sum()
    r['Country']=c
    final_df=pd.concat([final_df,r])

In [25]:
final_df

,C1_0,C1_1,C1_2,C1_3,C2_0,C2_1,C2_2,C2_3,C3_0,C3_1,...,I4LD_2,I4LD_3,I4LD_4,I4L_0,I4L_1,I4L_2,I4L_3,I4L_4,M1_0,Country
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-01,1,0,0,0,1,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1,AD
2020-01-02,1,0,0,0,1,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1,AD
2020-01-03,1,0,0,0,1,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1,AD
2020-01-04,1,0,0,0,1,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1,AD
2020-01-05,1,0,0,0,1,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1,AD
2020-01-06,1,0,0,0,1,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1,AD
2020-01-07,1,0,0,0,1,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1,AD
2020-01-08,1,0,0,0,1,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1,AD
2020-01-09,1,0,0,0,1,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1,AD


In [26]:
clean=final_df[final_df['Country']=='FR']
clean[clean['C1_2']!=0].index[0]

datetime.date(2020, 5, 11)

In [27]:
def policy_impl_date(df,ID,policy):
    try:
        clean=df[df['Country']==ID]
        result=clean[clean[policy]!=0].index[0]
    except:
        result=np.nan
    return result

In [28]:
policy_impl_date(final_df,'FR','C3_1')

datetime.date(2020, 7, 11)

In [29]:
policy_ID_list=list(p.TypeVal.unique())
country_ID_list=list(final_df.Country.unique())

In [30]:
Date=[]
Policy=[]
ID=[]
for country in country_ID_list:
    for policy in policy_ID_list:
        Policy.append(policy)
        ID.append(country)
        Date.append(policy_impl_date(final_df,country,policy))

In [31]:
baseline_df=pd.DataFrame()
baseline_df['Date']=Date
baseline_df['ID']=ID
baseline_df['Policy']=Policy

In [32]:
baseline_df.shape

(33125, 3)

In [33]:
baseline_df.to_csv('/Users/paulcharpentier/Desktop/Coronanet_files/new_baseline_df.csv')

In [34]:
#print(p_df.shape)
#p_df.head()

In [35]:
#p_df=p[['Country','Policy','PolicyValue','TypeVal']]
#clean_df=p_df[p_df['Policy']=='C1'][["Country"]]
#col_names=list(p_df.TypeVal.unique())
#
#for c in col_names:
#    clean_df[c]=p_df[p_df['TypeVal']==c].PolicyValue.values